In [1]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import math
pd.options.display.precision = 15

In [3]:
!ls ../input/champs-scalar-coupling

dipole_moments.csv		   structures
magnetic_shielding_tensors.csv	   structures.csv
mulliken_charges.csv		   structures.zip
potential_energy.csv		   test.csv
sample_submission.csv		   train.csv
scalar_coupling_contributions.csv


In [4]:
!conda install -y -c openbabel openbabel 
import openbabel


Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - openbabel


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |             main           3 KB
    conda-4.7.5                |           py37_0         3.0 MB
    conda-package-handling-1.3.10|           py37_0         259 KB
    libgcc-7.2.0               |       h69d50b8_2         304 KB
    openbabel-2.4.1            |           py37_5         5.1 MB  openbabel
    ------------------------------------------------------------
                                           Total:         8.6 MB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      pkgs/main/linux-64::_libgcc_mutex-0.1-main
  conda-package-han~ pkgs/main/linux-64::conda-package-handling-1.3.10-py37_0
  libgcc             pkgs/main/linux-64::libgcc-7.2.0-h69d50b8_2
  openb

In [25]:
import pandas as pd
import numpy as np
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [20]:
file_folder = '../input/champs-scalar-coupling'
train = pd.read_csv(f'{file_folder}/train.csv')
test = pd.read_csv(f'{file_folder}/test.csv')
structures = pd.read_csv('../input/champs-scalar-coupling/structures.csv')
x = structures.groupby('molecule_name').atom_index.max().reset_index(drop=False)
x.columns = ['molecule_name','totalatoms']
x.totalatoms+=1
train = train.merge(x,on='molecule_name')
#train = train[train.molecule_name=='dsgdb9nsd_000001']

In [21]:
obConversion = openbabel.OBConversion()
obConversion.SetInFormat("xyz")
structdir='../input/champs-scalar-coupling/structures/'
mols=[]
mols_files=os.listdir(structdir)
mols_index=dict(map(reversed,enumerate(mols_files)))
for f in mols_index.keys():
    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, structdir+f) 
    mols.append(mol)

In [27]:
t = train.groupby('molecule_name')

In [22]:
stats = []
for m,groupdf in tqdm(train.groupby('molecule_name')):
    mol=mols[mols_index[m+'.xyz']]
    for i in groupdf.index.values:
        totalatoms = groupdf.loc[i].totalatoms
        firstatomid = int(groupdf.loc[i].atom_index_0)
        secondatomid = int(groupdf.loc[i].atom_index_1)
        entrystats = {}
        entrystats['totalatoms'] = totalatoms
        entrystats['scalar_coupling_constant'] = float(groupdf.loc[i].scalar_coupling_constant)
        entrystats['type'] = groupdf.loc[i]['type']
        a = mol.GetAtomById(firstatomid)
        b = mol.GetAtomById(secondatomid)
        entrystats['molecule_name'] = m
        entrystats['atom_index_0'] = firstatomid
        entrystats['atom_index_1'] = secondatomid
        entrystats['bond_distance'] = a.GetDistance(b)
        entrystats['bond_atom'] = b.GetType()

        #Put the tertiary data in order of distance from first hydrogen
        tertiarystats = {}
        for j,c in enumerate(list(set(range(totalatoms)).difference(set([firstatomid,secondatomid])))):
            tertiaryatom = mol.GetAtomById(c)
            tp = tertiaryatom.GetType()
            dist = a.GetDistance(tertiaryatom)
            ang = a.GetAngle(b,tertiaryatom)*math.pi/180
            while(dist in tertiarystats):
                dist += 1e-15
                # print('Duplicates!',m,j,dist)
            tertiarystats[dist] = [tp,dist,ang]
        
        for k, c in enumerate(sorted(tertiarystats.keys())):
            entrystats['tertiary_atom_'+str(k)] = tertiarystats[c][0]
            entrystats['tertiary_distance_'+str(k)] = tertiarystats[c][1]
            entrystats['tertiary_angle_'+str(k)] = tertiarystats[c][2]
        stats.append(entrystats)
        

 68%|██████▊   | 57503/85003 [48:51<26:32, 17.27it/s]

KeyboardInterrupt: 

In [24]:
del stats
gc.collect()

3230

In [23]:
import gc

In [ ]:
obtrain = pd.DataFrame(stats)
obtrain.head(10)

In [ ]:
save_path = './../input/champs-scalar-coupling/structure_detail.csv'
obtrain.to_csv(save_path, index=False)